## dataLoad

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack
base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=2) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((2525956, 26), (2011, 26), (259370, 26))

((2525956, 25), (2525956, 1), (2011, 25), (2011, 1))

In [5]:
df[:2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,...,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind,solvesec,time_category,solvesec_3600
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,3,...,0.947683,0.222749,1268,0.955022,0.207410,637,train,3.0,1,3.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.0,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,8.0,3,8.0


In [6]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)


## 문제푼 시간 초로만 추가한 것

In [7]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model2, FEATS, 'LGBM_Baseline_문제푼시간초로만_timestamp시작시간고려.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057854 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.636889
[200]	valid_0's binary_logloss: 0.627956
[300]	valid_0's binary_logloss: 0.61702
[400]	valid_0's binary_logloss: 0.606031
[500]	valid_0's binary_logloss: 0.600917
[600]	valid_0's binary_logloss: 0.597458
[700]	valid_0's binary_logloss: 0.594846
[800]	valid_0's binary_logloss: 0.59095
[900]	valid_0's binary_logloss: 0.584513
[1000]	valid_0's binary_logloss: 0.58157
[1100]	valid_0's binary_logl

## 문제푼 시간 3600초 이상 모두 3600으로 변환

In [8]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초_timestamp시작시간고려.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234207 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.635652
[200]	valid_0's binary_logloss: 0.62286
[300]	valid_0's binary_logloss: 0.617181
[400]	valid_0's binary_logloss: 0.613279
[500]	valid_0's binary_logloss: 0.607887
[600]	valid_0's binary_logloss: 0.603502
[700]	valid_0's binary_logloss: 0.601559
[800]	valid_0's binary_logloss: 0.597103
[900]	valid_0's binary_logloss: 0.594014
[1000]	valid_0's binary_logloss: 0.591036
[1100]	valid_0's binary_logloss: 0.586717
[1200]	valid_0's binary_logloss: 0.584269
[1300]

### validation

In [115]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]
preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8194181098835474 ACC : 0.7370959961408586



In [49]:
lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv


## 문제푼 시간 카테고리 변수도 추가

In [9]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")
lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.637445
[200]	valid_0's binary_logloss: 0.629309
[300]	valid_0's binary_logloss: 0.62248
[400]	valid_0's binary_logloss: 0.613333
[500]	valid_0's binary_logloss: 0.60244
[600]	valid_0's binary_logloss: 0.595522
[700]	valid_0's binary_logloss: 0.592256
[800]	valid_0's binary_logloss: 0.5893
[900]	valid_0's binary_logloss: 0.586738
[1000]	valid_0's binary_logloss: 0.583592
[1100]	valid_0's binary_loglo

### validation

In [10]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category'
        ]
preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8120789220404234 ACC : 0.7265042267528593



In [18]:
post_slack("done")

lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')